In [1]:
import pandas as pd
import numpy as np 
import re
% matplotlib inline
import matplotlib.pyplot as plt
from collections import Counter

In [33]:
from sklearn.datasets import dump_svmlight_file
from sklearn.datasets import load_svmlight_file

In [2]:
import pickle
import time

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [4]:
clean_final = pd.read_csv("~/Documents/Capstone/code/clean_final_news.csv", encoding = "utf8", index_col = 0)

In [5]:
clean_final["binary_response"] = np.where(clean_final["response"] == "Not fake", 0, 1)

In [6]:
y_response = clean_final.binary_response.values

In [83]:
fileObject = open('token_unigram_text','rb')  
uni_lem = pickle.load(fileObject)  

In [127]:
fileObject.close()

In [84]:
real = uni_lem[0:23571]
fake = uni_lem[23571: ]

In [101]:
len(real)

23571

In [102]:
len(fake)

11040

In [85]:
real_list = [item for sublist in real for item in sublist]

In [86]:
fake_list = [item for sublist in fake for item in sublist]

In [87]:
fake_dict = Counter(fake_list)
real_dict = Counter(real_list)

In [115]:
word_count = pd.DataFrame([fake_dict, real_dict]).T

In [116]:
word_count.head()

,0,1
"""></span",4.0,NaN
"""frankly",1.0,NaN
"""full",1.0,NaN
"""he",1.0,NaN
"""here",1.0,NaN


In [117]:
word_count = word_count.fillna(0)

In [118]:
word_count.columns = ['fake', 'real']

In [119]:
word_count['fakepct'] = word_count['fake'].apply(lambda x: x / 11040)
word_count['realpct'] = word_count['real'].apply(lambda x: x / 23571)

In [120]:
word_count['fakepct'] = word_count['fakepct'].apply(lambda x: x + 1)
word_count['realpct'] = word_count['realpct'].apply(lambda x: x + 1)

In [121]:
word_count['fake2real'] = word_count['fakepct'] / word_count['realpct']
word_count['real2fake'] = word_count['realpct'] / word_count['fakepct']

### Most Fake Words

In [123]:
word_count.sort_values("fake2real", ascending = False).head(20)

,fake,real,fakepct,realpct,fake2real,real2fake
hillary,14339.0,4916.0,2.298822,1.208561,1.902115,0.525731
clinton,20680.0,12550.0,2.873188,1.532434,1.874918,0.533357
war,10452.0,4301.0,1.946739,1.182470,1.646333,0.607411
russia,7359.0,2662.0,1.666576,1.112935,1.497460,0.667798
fbi,6137.0,1837.0,1.555888,1.077935,1.443397,0.692810
email,8727.0,7023.0,1.790489,1.297951,1.379474,0.724914
russian,5207.0,2126.0,1.471649,1.090196,1.349894,0.740799
syria,4610.0,1343.0,1.417572,1.056977,1.341158,0.745625
military,5350.0,2955.0,1.484601,1.125366,1.319217,0.758026
obama,8450.0,8068.0,1.765399,1.342285,1.315219,0.760330


### Most Real Words

In [124]:
word_count.sort_values("real2fake", ascending = False).head(20)

,fake,real,fakepct,realpct,fake2real,real2fake
's,7704.0,55661.0,1.697826,3.361419,0.505092,1.979837
say,20428.0,90900.0,2.850362,4.856434,0.586925,1.703795
game,1267.0,17933.0,1.114764,1.760808,0.633098,1.579533
be,6196.0,34344.0,1.561232,2.457045,0.635410,1.573786
caption,73.0,12144.0,1.006612,1.515209,0.664339,1.505256
photo,1446.0,15009.0,1.130978,1.636757,0.690987,1.447205
usa,1014.0,13195.0,1.091848,1.559798,0.699993,1.428586
fullscreen,0.0,8856.0,1.000000,1.375716,0.726894,1.375716
team,1707.0,13540.0,1.154620,1.574435,0.733355,1.363596
company,2782.0,15815.0,1.251993,1.670952,0.749269,1.334634


### Most Frequent Fake Words

In [125]:
word_count.sort_values("real", ascending = False).head(20)

,fake,real,fakepct,realpct,fake2real,real2fake
say,20428.0,90900.0,2.850362,4.856434,0.586925,1.703795
's,7704.0,55661.0,1.697826,3.361419,0.505092,1.979837
trump,22159.0,49707.0,3.007156,3.108820,0.967298,1.033807
not,21009.0,44642.0,2.902989,2.893937,1.003128,0.996882
year,13517.0,43247.0,2.224366,2.834755,0.784677,1.274410
new,13020.0,38447.0,2.179348,2.631115,0.828298,1.207294
be,6196.0,34344.0,1.561232,2.457045,0.635410,1.573786
people,20483.0,29720.0,2.855344,2.260871,1.262940,0.791803
time,13641.0,28781.0,2.235598,2.221034,1.006557,0.993486
like,13150.0,25285.0,2.191123,2.072716,1.057126,0.945961


### Most Frequent Real Words

In [126]:
word_count.sort_values("fake", ascending = False).head(20)

,fake,real,fakepct,realpct,fake2real,real2fake
trump,22159.0,49707.0,3.007156,3.108820,0.967298,1.033807
not,21009.0,44642.0,2.902989,2.893937,1.003128,0.996882
clinton,20680.0,12550.0,2.873188,1.532434,1.874918,0.533357
people,20483.0,29720.0,2.855344,2.260871,1.262940,0.791803
say,20428.0,90900.0,2.850362,4.856434,0.586925,1.703795
state,14936.0,24048.0,2.352899,2.020237,1.164665,0.858616
hillary,14339.0,4916.0,2.298822,1.208561,1.902115,0.525731
time,13641.0,28781.0,2.235598,2.221034,1.006557,0.993486
year,13517.0,43247.0,2.224366,2.834755,0.784677,1.274410
like,13150.0,25285.0,2.191123,2.072716,1.057126,0.945961


In [45]:
cv = CountVectorizer()
cv2 = CountVectorizer()
real_cv = cv.fit_transform(real)
fake_cv = cv2.fit_transform(fake)

In [46]:
cv.vocabulary_

{'salary_debt_ratio': 174668,
 'caribbean_sun': 34089,
 'leave_intact': 114773,
 'zach_sanford': 223180,
 'murthys': 134158,
 'hakone': 87866,
 'rehabilitator': 166289,
 'advertised': 7769,
 'revolving_door': 169140,
 'strangest': 192299,
 'rsr': 172480,
 'reckon': 165089,
 'scalia_die': 176394,
 'fail_miserably': 70562,
 'jesse_ventura': 104381,
 'neha': 136851,
 'pre_recession': 156538,
 'east_africa': 62778,
 'todenhoefer': 202193,
 'widely_know': 218362,
 'originally_slovakia': 144670,
 'nuthrax': 141172,
 'rooftops': 171631,
 'gj9c0klnda': 82750,
 'technographics': 198159,
 'saudi_arabia_ally': 176002,
 'corset': 47162,
 'hurdlers': 95583,
 'realtys': 164518,
 'outlive': 145380,
 'huasteca': 94943,
 'mr_trumps': 133330,
 'autoregulate': 18063,
 'vaalbeek': 211115,
 'long_losing_streak': 118634,
 'economy_grow_percent': 63355,
 'self_conscious': 179072,
 'canfora': 33233,
 'pachyderm': 146438,
 'nihang': 138671,
 'khans': 109112,
 'intervener': 100718,
 'fuzzie': 79338,
 'osvaldo_d

In [36]:
text_tfidf[0]

<1x272889 sparse matrix of type '<class 'numpy.float64'>'
	with 324 stored elements in Compressed Sparse Row format>

In [ ]:
dense_text = text_tfidf.todense()

In [31]:
dump_svmlight_file(text_tfidf, y_response, 'tfidf.dat')

In [34]:
data = load_svmlight_file("tfidf.dat")

In [35]:
data[0]

<34611x272889 sparse matrix of type '<class 'numpy.float64'>'
	with 7058044 stored elements in Compressed Sparse Row format>

In [37]:
def get_data():
    data = load_svmlight_file("tfidf.dat")
    return data[0], data[1]

X, y = get_data()

In [39]:
X.todense()

MemoryError: 

In [9]:
# Get list of all words in tfidf vocabulary
features = tfidf.get_feature_names()

In [22]:
features[6000:6100]

['_polk_giant',
 '_porchon',
 '_porchon_lynch',
 '_post',
 '_post86694',
 '_post_facebook',
 '_pound',
 '_pound_turf',
 '_preoccupy',
 '_presidency_potus',
 '_president',
 '_president_ail',
 '_president_barack',
 '_president_george',
 '_presidential_election',
 '_presidential_race',
 '_prime',
 '_prime_minister_theresa',
 '_programs',
 '_prole_iowa_charlie',
 '_prosise',
 '_pryor',
 '_pst',
 '_pt',
 '_public_schools',
 '_purcell',
 '_purchases',
 '_puskar',
 '_puskar_ap',
 '_putin',
 '_putins',
 '_qamar',
 '_qamar_javed',
 '_quarter',
 '_quarter_ford_field',
 '_question',
 '_raheel_sharif',
 '_rajendar',
 '_rally_boulder',
 '_ramon',
 '_ramon_espinosa_ap',
 '_rams',
 '_rand',
 '_rand_paul',
 '_rand_paul_r',
 '_ranger',
 '_raul',
 '_rebilas_usa',
 '_red_wings',
 '_redick',
 '_reduction',
 '_referendum',
 '_refugee',
 '_refugee_agency',
 '_region',
 '_regis',
 '_renewable',
 '_renewable_energy',
 '_rep',
 '_reporting',
 '_reporting_washington',
 '_representative_tom',
 '_republican_estab

In [23]:
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

In [24]:
def top_overall_words(text, response, features, cls=None, min_tfidf=0.1, top_n=25):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in ids. '''
    if cls:
        ids = np.where(response == cls)
        D = text[ids].toarray()
    else:
        D = text.toarray()

    D[D < min_tfidf] = 0 # so common words across all documents won't dominate - may remove
    tfidf_mean = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_mean, features, top_n)

In [25]:
top_overall_words(text_tfidf, y_response, features)

MemoryError: 

In [ ]:
#def plot_tfidf_classfeats_h(dfs):
#    ''' Plot the data frames returned by the function plot_tfidf_classfeats(). '''
#    fig = plt.figure(figsize=(12, 9), facecolor="w")
#    x = np.arange(len(dfs[0]))
#    for i, df in enumerate(dfs):
#        ax = fig.add_subplot(1, len(dfs), i+1)
#        ax.spines["top"].set_visible(False)
#        ax.spines["right"].set_visible(False)
#        ax.set_frame_on(False)
#        ax.get_xaxis().tick_bottom()
#        ax.get_yaxis().tick_left()
#        ax.set_xlabel("Mean Tf-Idf Score", labelpad=16, fontsize=14)
#        ax.set_title("label = " + str(df.label), fontsize=16)
#        ax.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
#        ax.barh(x, df.tfidf, align='center', color='#3F5D7D')
#        ax.set_yticks(x)
#        ax.set_ylim([-1, x[-1]+1])
#        yticks = ax.set_yticklabels(df.feature)
#        plt.subplots_adjust(bottom=0.09, right=0.97, left=0.15, top=0.95, wspace=0.52)
#    plt.show()